<a href="https://colab.research.google.com/github/Mikimorka/Test/blob/main/Recenzie_uk%C3%A1%C5%BEka.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# Importujeme potrebné knižnice
# ---

# Pandas pre prácu s dátami v tabuľkovom formáte (ako Excel)
import pandas as pd
# TfidfVectorizer na premenu textu na čísla (pre strojové učenie)
from sklearn.feature_extraction.text import TfidfVectorizer
# train_test_split na rozdelenie dát pre trénovanie a testovanie
from sklearn.model_selection import train_test_split
# LogisticRegression ako jednoduchý, ale efektívny model pre klasifikáciu
from sklearn.linear_model import LogisticRegression
# classification_report na vyhodnotenie, ako dobre si model viedol
from sklearn.metrics import classification_report

# 1. Príprava dát (simulujeme, že sme ich načítali z CSV alebo iného zdroja)
# ---
print("--- Krok 1: Príprava dát ---")
data = pd.DataFrame({
    'recenzia': [
        "Tento produkt je úžasný, som nadšený!",        # Pozitívna
        "Naozaj skvelý a funguje perfektne.",             # Pozitívna
        "Úplne nespokojný, strata peňazí.",              # Negatívna
        "Odporúčam, veľmi dobrá kvalita.",                # Pozitívna
        "Katastrofa, nikdy viac!",                        # Negatívna
        "Celkom fajn, ale čakal som viac.",               # Neutrálna/mierne negatívna
        "Super rýchle dodanie a top kvalita.",             # Pozitívna
        "Veľmi spokojný s nákupom.",                      # Pozitívna
        "Vôbec mi to nepomohlo.",                         # Negatívna
        "Je to v poriadku, nič extra.",                   # Neutrálna
        "Najlepšie, čo som kedy kúpil!",                 # Pozitívna
        "Úplne sklamaný z výsledku.",                     # Negatívna
        "Funguje, ako má, som rád.",                      # Pozitívna
        "Mrzí ma, že som si to kúpil.",                   # Negatívna
        "Celkom dobrý pomer cena/výkon.",                 # Pozitívna/Neutrálna
        "Veľmi dobrá skúsenosť, odporúčam.",             # Pozitívna
        "Nefunguje to, ako som očakával.",                # Negatívna
        "Spokojnosť, splnilo to účel.",                   # Pozitívna
        "Najhorší nákup vôbec.",                          # Negatívna
        "Som nadšený z tohto produktu.",                  # Pozitívna
        "Absolútne nepoužiteľné.",                        # Negatívna
        "Funguje presne podľa popisu.",                   # Pozitívna
        "Veľmi sklamaný z dodania.",                      # Negatívna
        "Skvelá hodnota za peniaze.",                     # Pozitívna
        "Neodporúčam, strata času.",                      # Negatívna
        "Celkom dobré, ale sú aj lepšie.",                # Neutrálna
        "Perfektné! Určite kúpim znova.",                 # Pozitívna
        "Veľmi nespokojný s kvalitou.",                   # Negatívna
        "Presne to, čo som potreboval.",                  # Pozitívna
        "Úplne zbytočné.",                                # Negatívna
        "Funguje bez problémov.",                         # Pozitívna
        "Veľké sklamanie.",                              # Negatívna
        "Spokojný s výsledkom.",                          # Pozitívna
        "Nepáči sa mi to.",                               # Negatívna
        "Výborný produkt, rýchle dodanie.",              # Pozitívna
        "Nekvalitné spracovanie.",                        # Negatívna
        "Funguje, ako má.",                               # Pozitívna
        "Veľmi nápomocné.",                              # Pozitívna
        "Úplne sklamaný.",                                # Negatívna
        "Maximálna spokojnosť.",                          # Pozitívna
        "Neodporúčam tento predajcu.",                    # Negatívna
        "Funguje, ale pomaly.",                           # Neutrálna/Negatívna
        "Skvelá podpora zákazníkov.",                     # Pozitívna
        "Problémy s inštaláciou.",                       # Negatívna
        "Veľmi užitočné.",                                # Pozitívna
    ],
    'sentiment': [1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1] # 1 = pozitívny, 0 = negatívny/neutrálny
})
print("Ukážka našich recenzií:")
print(data.head()) # Zobrazíme len prvých pár riadkov, keďže ich je viac
print("\n")

# 2. Predspracovanie textu a premena na číselné príznaky (TF-IDF)
# ---
print("--- Krok 2: Premena textu na čísla (Feature Engineering) ---")
# TF-IDF (Term Frequency-Inverse Document Frequency)
# Prevedie text na číselné vektory. Každé slovo dostane váhu
# podľa toho, ako často sa vyskytuje v recenzii a ako zriedkavo v celom súbore.
# max_features=100 znamená, že zohľadníme len 100 najdôležitejších slov.
vectorizer = TfidfVectorizer(max_features=100, stop_words=['je', 'a', 'som', 'ale'])
# .fit_transform() sa naučí slovník z dát a zároveň ich prevedie na vektory
X = vectorizer.fit_transform(data['recenzia'])

# Zobrazenie, aké slová náš vektorizátor "vidí" (premenné/príznaky)
print("Slová (príznaky), ktoré model z textu extrahoval:")
print(vectorizer.get_feature_names_out())
print(f"Tvar našich číselných dát (recenzie x slová): {X.shape}\n")


# 3. Rozdelenie dát na trénovaciu a testovaciu sadu
# ---
print("--- Krok 3: Rozdelenie dát na trénovanie a testovanie ---")
# test_size=0.2 na rozdelenie dát (80% trénovanie, 20% testovanie)
# random_state zabezpečí, že výsledok bude vždy rovnaký pri každom spustení.
X_train, X_test, y_train, y_test = train_test_split(X, data['sentiment'], test_size=0.2, random_state=42)

print(f"Počet trénovacích recenzií: {X_train.shape[0]}")
print(f"Počet testovacích recenzií: {X_test.shape[0]}\n")

# 4. Trénovanie modelu strojového učenia
# ---
print("--- Krok 4: Trénovanie modelu ---")
# LogisticRegression je jednoduchý, ale efektívny algoritmus pre klasifikáciu.
# Aj keď má v názve 'Regression', je to primárne klasifikačný model.
model = LogisticRegression()
# .fit() je metóda, ktorá "učí" model na trénovacích dátach (X_train) a ich správnych odpovediach (y_train).
model.fit(X_train, y_train)
print("Model bol úspešne natrénovaný.\n")

# 5. Vyhodnotenie modelu na testovacích dátach
# ---
print("--- Krok 5: Vyhodnotenie modelu ---")
# .predict() použijeme na to, aby model predpovedal sentiment pre dáta, ktoré ešte nevidel (testovacie dáta).
y_pred = model.predict(X_test)

# classification_report nám dáva prehľad o presnosti, precíznosti, návratnosti a F1-skóre.
# Povie nám, ako dobre model rozlíšil pozitívne (1) a negatívne/neutrálne (0) recenzie.
print("Správa o vyhodnotení modelu:")
print(classification_report(y_test, y_pred))
print("\n")

# 6. Predikcia pre novú recenziu
# ---
print("--- Krok 6: Predikcia pre novú recenziu ---")
nova_recenzia = "Vôbec sa mi to nepáči, som veľmi sklamaný."
# Novú recenziu musíme premeniť na čísla ROVNAKÝM vektorizátorom,
# ktorý sa naučil slovník z trénovacích dát!
nova_recenzia_features = vectorizer.transform([nova_recenzia])

# Model predpovedá sentiment. .predict() vráti 0 alebo 1.
predikcia = model.predict(nova_recenzia_features)[0]

sentiment_map = {0: "Negatívny/Neutrálny", 1: "Pozitívny"}

print(f"Nová recenzia: '{nova_recenzia}'")
print(f"Model predikuje sentiment: {sentiment_map[predikcia]}")

--- Krok 1: Príprava dát ---
Ukážka našich recenzií:
                                recenzia  sentiment
0  Tento produkt je úžasný, som nadšený!          1
1     Naozaj skvelý a funguje perfektne.          1
2       Úplne nespokojný, strata peňazí.          0
3        Odporúčam, veľmi dobrá kvalita.          1
4                Katastrofa, nikdy viac!          0


--- Krok 2: Premena textu na čísla (Feature Engineering) ---
Slová (príznaky), ktoré model z textu extrahoval:
['absolútne' 'aj' 'ako' 'bez' 'celkom' 'cena' 'dobrá' 'dobré' 'dobrý'
 'dodania' 'dodanie' 'extra' 'fajn' 'funguje' 'hodnota' 'inštaláciou'
 'katastrofa' 'kedy' 'kvalita' 'kvalitou' 'kúpil' 'kúpim' 'lepšie' 'ma'
 'maximálna' 'mi' 'mrzí' 'má' 'nadšený' 'najhorší' 'najlepšie' 'naozaj'
 'nefunguje' 'nekvalitné' 'neodporúčam' 'nepomohlo' 'nepoužiteľné'
 'nepáči' 'nespokojný' 'nikdy' 'nič' 'nákup' 'nákupom' 'nápomocné'
 'odporúčam' 'očakával' 'peniaze' 'perfektne' 'perfektné' 'peňazí'
 'podpora' 'podľa' 'pomaly' 'pomer' '